### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

AttributeError: ignored

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [0]:
#You can use wget to download the file directly
!wget http://www.manythings.org/anki/hin-eng.zip --quiet

In [0]:
!ls -l

In [0]:
!unzip hin-eng.zip

In [0]:
!cat hin.txt

In [0]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [0]:
len(data)

In [0]:
data[40000:50000]


### Extract Source and Target Language pairs

In [0]:
#Split by newline character
data =  data.split('\n')

#Show some Data
data[100:105]

In [0]:
len(data)

### Separate Source and Target pairs

In [0]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt,_ = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

### Separate Source and Target pairs..

In [0]:
encoder_text[100:105]

In [0]:
decoder_text[100:105]

### Tokenize Source language sentences

In [0]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer()
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences

In [13]:
len(encoder_t.word_index)

2376

In [14]:
encoder_t.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'you': 4,
 'is': 5,
 'a': 6,
 'he': 7,
 'of': 8,
 'in': 9,
 'my': 10,
 'it': 11,
 'me': 12,
 'this': 13,
 'have': 14,
 'she': 15,
 'was': 16,
 'for': 17,
 'are': 18,
 'do': 19,
 'that': 20,
 'his': 21,
 'your': 22,
 'will': 23,
 'what': 24,
 'on': 25,
 'we': 26,
 "don't": 27,
 'at': 28,
 'him': 29,
 'her': 30,
 'not': 31,
 'like': 32,
 'go': 33,
 'with': 34,
 'be': 35,
 'how': 36,
 'and': 37,
 'can': 38,
 "i'm": 39,
 'has': 40,
 'there': 41,
 'time': 42,
 'know': 43,
 'all': 44,
 'up': 45,
 'they': 46,
 'come': 47,
 'very': 48,
 'as': 49,
 'please': 50,
 'had': 51,
 'from': 52,
 "it's": 53,
 'by': 54,
 'out': 55,
 'am': 56,
 'want': 57,
 'when': 58,
 'did': 59,
 'here': 60,
 'no': 61,
 'been': 62,
 "can't": 63,
 'going': 64,
 'get': 65,
 'take': 66,
 'an': 67,
 'father': 68,
 'book': 69,
 'about': 70,
 'if': 71,
 'one': 72,
 'were': 73,
 'money': 74,
 'india': 75,
 'would': 76,
 'tom': 77,
 'long': 78,
 'today': 79,
 'where': 80,
 'two': 81,
 'but': 82,
 

In [15]:
#Convert English text to indexes
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers 

encoder_seq[100:105] #Display some converted sentences

[[2, 27, 43], [2, 27, 43], [2, 14, 6, 97], [2, 14, 6, 124], [2, 209]]

In [16]:
encoder_text[100:105]

["I don't know.",
 "I don't know.",
 'I have a car.',
 'I have a dog.',
 'I understand.']

In [17]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Maximum sentence length for Source language:  22
Source language vocablury size:  2376


### Tokenize Target language sentences

In [0]:
?tf.keras.preprocessing.text.Tokenizer

In [0]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [19]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  2974


In [20]:
decoder_t.word_index

{'<start>': 1,
 '<end>': 2,
 'है।': 3,
 'में': 4,
 'नहीं': 5,
 'मैं': 6,
 'वह': 7,
 'से': 8,
 'के': 9,
 'मुझे': 10,
 'क्या': 11,
 'है': 12,
 'को': 13,
 'हैं।': 14,
 'की': 15,
 'हो': 16,
 'बहुत': 17,
 'का': 18,
 'उसने': 19,
 'पर': 20,
 'एक': 21,
 'हूँ।': 22,
 'तुम': 23,
 'था।': 24,
 'यह': 25,
 'कर': 26,
 'लिए': 27,
 'मेरे': 28,
 'कि': 29,
 'और': 30,
 'उसे': 31,
 'इस': 32,
 'हैं': 33,
 'भी': 34,
 'अपने': 35,
 'मेरी': 36,
 'मैंने': 37,
 'रहा': 38,
 'ने': 39,
 'पास': 40,
 'तो': 41,
 'अपनी': 42,
 'करने': 43,
 'गया।': 44,
 'कल': 45,
 'ही': 46,
 'हम': 47,
 'काम': 48,
 'कोई': 49,
 'उसके': 50,
 'तुम्हारे': 51,
 'उसकी': 52,
 'था': 53,
 'साथ': 54,
 'घर': 55,
 'तुम्हें': 56,
 'करना': 57,
 'उस': 58,
 'थी।': 59,
 'मेरा': 60,
 'गया': 61,
 'सकते': 62,
 'समय': 63,
 'बात': 64,
 'रही': 65,
 'पता': 66,
 'आप': 67,
 'कुछ': 68,
 'आज': 69,
 'तुम्हे': 70,
 'उसको': 71,
 'चाहिए।': 72,
 'यहाँ': 73,
 'अभी': 74,
 'बजे': 75,
 'तक': 76,
 'लगता': 77,
 'किताब': 78,
 'मुझसे': 79,
 'पसंद': 80,
 'अच्छा': 81,
 'दिया।': 82,

### Compare different sentences length

In [0]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

In [0]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

### How do we make it same?

### Padding the sentences

In [0]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')

In [22]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2785, 22)
Target data shape:  (2785, 27)


In [0]:
encoder_text[200]

In [0]:
encoder_input_data[200]

In [0]:
decoder_text[200]

In [0]:
decoder_input_data[200]

#### Integer to Word converter for Decoder data

In [0]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [24]:
int_to_word_decoder[15]

'की'

### Building Decoder Output

In [25]:
decoder_input_data.shape

(2785, 27)

In [0]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], 
                                decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

In [0]:
#decoder_t.word_index

In [0]:
#<start> yeh kitab hai <end>
#Yeh kitab hai <end> 0

In [27]:
decoder_input_data[0]

array([  1, 752,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

In [28]:
decoder_target_data[0]

array([752.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

#### Convert target data in one hot vector

In [0]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [30]:
decoder_target_one_hot.shape

(2785, 27, 2975)

In [0]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)    

In [0]:
decoder_target_one_hot.shape

In [0]:
decoder_target_one_hot[0].shape

### Building the Training Model

In [0]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

#### Build Encoder

In [0]:
tf.keras.backend.clear_session()

In [34]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(None,))

#Embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, 
                                              encoder_embedding_size)

#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#LSTM Layer and its output
x, state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)

#Build a list to feed Decoder - Sentence Embedding
encoder_states = [state_h, state_c]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
encoder_embedding_output

#### Build Decoder

In [0]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(None,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, 
                                              decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(rnn_units, 
                                   return_sequences=True, 
                                   return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
x,_,_ = decoder_rnn(decoder_embedding_output, 
                    initial_state=encoder_states)

#Output Layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, #+1 to make sure one-hot encoding works for highest index value
                                      activation='softmax')

#Output of Dense layer
decoder_outputs = decoder_dense(x)

In [0]:
x

In [0]:
decoder_outputs

### Build Model using both Encoder and Decoder

In [0]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [37]:
model.output

<tf.Tensor 'dense/truediv:0' shape=(?, ?, 2975) dtype=float32>

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

### Train the model

In [39]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     118850      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     148750      input_2[0][0]                    
______________________________________________________________________________________________

In [40]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_one_hot,
          batch_size=64,
          epochs=20,
          validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2228 samples, validate on 557 samples
Epoch 1/20
2228/2228 [==============================] - 33s 15ms/sample - loss: 3.7562 - val_loss: 2.9727
Epoch 2/20
2228/2228 [==============================] - 31s 14ms/sample - loss: 1.6351 - val_loss: 2.9699
Epoch 3/20
2228/2228 [==============================] - 31s 14ms/sample - loss: 1.5915 - val_loss: 3.0044
Epoch 4/20
2228/2228 [==============================] - 33s 15ms/sample - loss: 1.5544 - val_loss: 3.0222
Epoch 5/20
2228/2228 [==============================] - 31s 14ms/sample - loss: 1.5034 - val_loss: 2.8632
Epoch 6/20
2228/2228 [==============================] - 30s 13ms/sample - loss: 1.4410 - val_loss: 2.7229
Epoch 7/20
2228/2228 [==============================] - 30s 13ms/sample - loss: 1.3990 - val_loss: 2.7271
Epoch 8/20
2228/2228 [==============================] - 30s 13ms/sample - loss: 1.3702 - val_loss: 2.7269
Epoch 9/20


### Save the model for later reuse

In [0]:
model.save('seq2seq_training_translation.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [0]:
encoder_model = tf.keras.models.Model(encoder_inputs, #Padded input sequences
                                      encoder_states) #Hidden state and Cell state at last time step

In [0]:
encoder_model.output

### Build the Decoder Model 
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li>Get Decoder Dense layer output</li>
        <li>Build Model</li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [0]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Get Decoder RNN outputs along with h and c state

In [0]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 3 - Get Decoder Dense layer output

In [0]:
decoder_outputs = decoder_dense(rnn_outputs)

##### Step 4 - Build Decoder Model

In [0]:
decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                                      [decoder_outputs] + decoder_states)

# Predicting output from Seq2Seq model

##### Build a prediction function

In [0]:
decoder_t.word_index['<start>']

In [0]:
int_to_word_decoder[1]

In [0]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values - Sentence embedding
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    num_of_predictions = 0
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter index
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or num_of_predictions > max_decoder_seq_length):
            
            stop_loop = True
            continue
        
        num_of_predictions += 1
        
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

##### Call Prediction function on a random sentence

In [47]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 5):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

--------
Input sentence:  He has many history books.
Predicted sentence:  मुझे एक एक लिए बहुत नहीं है।
--------
Input sentence:  He hesitated for a moment.
Predicted sentence:  मुझे बहुत बहुत है।
--------
Input sentence:  He is one of my neighbors.
Predicted sentence:  मुझे अपने एक लिए बहुत नहीं है।
--------
Input sentence:  He likes his school a lot.
Predicted sentence:  मुझे एक एक लिए बहुत नहीं है।
--------
Input sentence:  He makes fun of everybody.
Predicted sentence:  मुझे एक बहुत बहुत है।


##### Save encoder and decoder model

In [0]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('seq2seq_decoder_eng_hin.hd5')  #Decoder model

##### Save encoder and decoder tokenizers

In [0]:
import pickle

pickle.dump(encoder_t,open('encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('decoder_tokenizer_hin','wb'))